In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import time

In [2]:
import pathlib
dataset = pathlib.Path(r"C:\Users\lulu5\Documents\echantillon_1")
image_count = len(list(dataset.glob('*/*.jpg')))
print(image_count)

15603


In [3]:
batch_size = 128
img_height = 224
img_width = 224
number_class = 1081

In [4]:
epochs = 10

# Unoptimized

In [5]:
TFs = {#'height_shift_range':  .5,
       "horizontal_flip": True,
       "vertical_flip": True,
       "rotation_range": 30,
       "featurewise_std_normalization": True,
       "brightness_range": (1, 1.5),
       "shear_range":0.2
       # other transformation your want
       # ...
       }
datagen = ImageDataGenerator(**TFs, validation_split=0.2)

C:\Users\lulu5\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [6]:
train_generator = datagen.flow_from_directory(
    dataset,
    color_mode="rgb",
    batch_size=batch_size,
    subset = "training",
    target_size=(img_height, img_width),
    shuffle=True,
    class_mode='binary')

validation_generator = datagen.flow_from_directory(
    dataset,
    color_mode="rgb",
    batch_size=batch_size,
    subset = "validation",
    target_size=(img_height, img_width),
    shuffle=True,
    class_mode='binary')

Found 12832 images belonging to 1081 classes.
Found 2771 images belonging to 1081 classes.


In [7]:
model_unoptimized = tf.keras.applications.resnet50.ResNet50(
    include_top=True, weights=None, input_tensor=None,
    input_shape=None, pooling=None, classes=number_class)

model_unoptimized.compile(optimizer="Adam", 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy', 'sparse_top_k_categorical_accuracy'])

In [8]:
start_time = time.perf_counter()
model_unoptimized.fit(train_generator, validation_data=validation_generator,epochs=epochs,batch_size=batch_size, verbose=1, steps_per_epoch=3,validation_steps=1)
print("Execution time:", time.perf_counter() - start_time)

C:\Users\lulu5\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\lulu5\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/10


C:\Users\lulu5\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


3/3 [==============================] - 124s 41s/step - loss: 8.3446 - accuracy: 0.0052 - sparse_top_k_categorical_accuracy: 0.0104 - val_loss: 791.2019 - val_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 0.8828
Epoch 2/10
3/3 [==============================] - 122s 42s/step - loss: 7.8638 - accuracy: 0.0052 - sparse_top_k_categorical_accuracy: 0.0234 - val_loss: 15948.5312 - val_accuracy: 0.0000e+00 - val_sparse_top_k_categorical_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 135s 46s/step - loss: 7.2311 - accuracy: 0.0052 - sparse_top_k_categorical_accuracy: 0.0182 - val_loss: 9995.0254 - val_accuracy: 0.0156 - val_sparse_top_k_categorical_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 110s 37s/step - loss: 6.5350 - accuracy: 0.0052 - sparse_top_k_categorical_accuracy: 0.0312 - val_loss: 1513.8801 - val_accuracy: 0.0156 - val_sparse_top_k_categorical_accuracy: 1.0000
Epoch 5/10
1/3 [=========>....................] - ETA: 1:08 - l

# Optimized

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

model_V1_augmented = Sequential()
model_V1_augmented.add(data_augmentation)
model = tf.keras.applications.resnet50.ResNet50(include_top=True, weights=None, input_tensor=None,input_shape=None, pooling=None, classes=number_class)
extracted_layer= Model(inputs = model.input, outputs=model.get_layer("avg_pool").output, name="model_V1")
extracted_layer.trainable = False
model_V1_augmented.add(extracted_layer)

model_V1_augmented.compile(optimizer="Adam", 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy', 'sparse_top_k_categorical_accuracy'])

In [ ]:
start_time = time.perf_counter()
model_V1_augmented.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  steps_per_epoch=3
)
print("Execution time:", time.perf_counter() - start_time)